<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Library installation

Install the required Python packages so the notebook runs end-to-end in a fresh environment. This ensures everyone tests the same versions when measuring a small calendar edge.

In [ ]:
!pip install yfinance pandas numpy matplotlib

These wheels install quickly on standard CPython via pip. For reproducibility, consider pinning versions in a requirements.txt when you move from exploration to backtesting. No non-standard system dependencies are needed for this notebook.

## Imports and setup

We import matplotlib.pyplot for plotting, pandas for time‑series manipulation, numpy for vectorized math and log returns, and yfinance to download historical TLT prices. This minimal stack keeps the focus on testing a calendar hypothesis rather than tooling.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import yfinance as yf

Keeping the stack small reduces chances of hidden defaults that skew results. Matplotlib’s stateful interface will render plots at the end when we call plt.show(), which keeps notebooks tidy. yfinance retrieves split/dividend-adjusted series we will treat as our baseline.

## Download data and build features

Fetch daily TLT prices from Yahoo Finance to cover two decades of regimes for a robust seasonality check. The long sample lets us see whether month-end strength persists across different rate cycles.

In [ ]:
tlt = yf.download("TLT", start="2002-01-01", end="2022-06-30")

Yahoo’s API returns a DataFrame with Open/High/Low/Close/Adj Close and a DateTime index. Using ETF data avoids survivorship bias from delisted bonds, though vendor adjustments can differ from your broker. Treat this as a quick scan; you can swap in official vendor data once the signal earns more work.

Compute daily log returns and attach calendar labels (day of month and year), then aggregate mean return by calendar day. This frames the problem as an event study across months.

In [ ]:
tlt["log_return"] = np.log(tlt["Adj Close"] / tlt["Adj Close"].shift(1))
tlt["day_of_month"] = tlt.index.day
tlt["year"] = tlt.index.year

In [ ]:
grouped_by_day = tlt.groupby("day_of_month")["log_return"].mean()

Log returns add across time and handle compounding cleanly, which makes later cumulation and comparisons less fragile. Grouping by the calendar day gives us a cross-month average without peeking into the future because each day uses only its own prior close. Short and long months contribute whatever days they have, so the average reflects actual trading calendars rather than an imposed template.

## Define month-end strategy proxy

Build a simple diagnostic spread: sum of returns in the last calendar week minus the first calendar week. It approximates buying into month-end strength and giving it back after the turn.

In [ ]:
tlt["first_week_returns"] = 0.0
tlt.loc[tlt.day_of_month <= 7, "first_week_returns"] = tlt[
    tlt.day_of_month <= 7
]["log_return"]

In [ ]:
tlt["last_week_returns"] = 0.0
tlt.loc[tlt.day_of_month >= 23, "last_week_returns"] = tlt[
    tlt.day_of_month >= 23
]["log_return"]

In [ ]:
tlt["last_week_less_first_week"] = (
    tlt["last_week_returns"] - tlt["first_week_returns"]
)

This is not an executable PnL yet; it’s a clean indicator of relative strength around the boundary. The definitions (<=7 and >=23) sidestep exact month lengths and exchange holidays while keeping the effect focused. If the spread is positive on average and reasonably stable, it justifies deeper work with precise trading days and costs.

## Visualize results and check stability

Chart average log returns by calendar day to see where any excess clusters. Visual inspection quickly tells us whether the edge lives near the end or start.

In [ ]:
grouped_by_day.plot.bar(
    title="Mean Log Returns by Calendar Day of Month"
)

Bars near the final trading days should stand out if flows lift prices into close, while early-month bars may dip if that strength mean-reverts. Plotting the average by day is the fastest way to validate the hypothesis before building a backtester. If nothing shows here, a more complex model probably will not rescue it.

Evaluate persistence with three views: yearly average of the spread, yearly cumulative contribution, and the full-sample cumulative path.

In [ ]:
(
    tlt.groupby("year")
    .last_week_less_first_week.mean()
    .plot.bar(title="Mean Log Strategy Returns by Year")
)

In [ ]:
(
    tlt.groupby("year")
    .last_week_less_first_week.sum()
    .cumsum()
    .plot(title="Cumulative Sum of Returns By Year")
)

In [ ]:
tlt["last_week_less_first_week"].cumsum().plot(
    title="Cumulative Sum of Returns By Day"
)

In [ ]:
plt.show()

Consistent positive bars by year indicate the behavior isn’t just a couple of lucky months. Yearly cumulative sums highlight which regimes drive or fade the edge, and the daily cumulative path reveals drawdowns you would have felt. We still ignore trading costs and ETF tracking error here; if the shape looks good, add slippage and calendars next.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.